****
# **DSA - II PROJECT - Secure File Transfer**
****

### **Merkle tree for testing file integrity**

***Submitted by:*** 

***-> CB.EN.U4AIE22101 - Manasha A*** 

***-> CB.EN.U4AIE22150 - Samhitha S***


***Merkle tree logic:***

- **Chunking and Hashing:** The input data is split into chunks, and the SHA-256 hash is computed for each chunk. This ensures that the input data is processed in manageable pieces.

- **Tree Construction:** The function then enters a loop where it continues to build the Merkle tree. In each iteration, it checks whether the number of hashes is odd. If odd, it duplicates the last hash to make it even.

- **Pairing and Concatenation:** The function then pairs and concatenates adjacent hashes to create a new list. This process is repeated until only one hash remains, which represents the root of the Merkle tree.

- **Decoding and Return:** The final hash is decoded from bytes to a string and returned as the Merkle tree root.


***Test files :***
<ul><li>ClientHere.txt</li>
<li>ClientHere.csv</li>
<li>FPR1_SSTR1_gene.fasta</li>
<li>SHA256.txt</li></ul>

In [10]:
import socket
import hashlib

def generate_merkle_tree(data):
    # Convert the input data into chunks and compute SHA-256 hashes for each chunk
    hash_list = [hashlib.sha256(chunk).hexdigest().encode() for chunk in data.split()]

    while len(hash_list) > 1:   # Iterating until there is only one hash remaining in the list
        if len(hash_list) % 2 != 0: # Checking if the number of hashes is odd, If odd, duplicate the last hash in the list
            hash_list.append(hash_list[-1])
        # Create a new list by pairing and concatenating adjacent hashes
        hash_list = [hashlib.sha256(hash_list[i] + hash_list[i + 1]).hexdigest().encode() for i in range(0, len(hash_list), 2)]
    return hash_list[0].decode()   # The final remaining hash is the root of the Merkle tree

def simple_encrypt(data):
    key = 0x5A
    return bytes([char ^ key for char in data])

def client_pgm():
    host = socket.gethostname()
    port = 9997

    client_socket = socket.socket()
    client_socket.connect((host, port))

    with open('C:/Users/manas/Desktop/test/FPR1_SSTR1_gene.fasta', 'r') as file:
        data = file.read()

    #print("Original data: ")
    #print(data)

    encrypted_data = simple_encrypt(data.encode())
    print("Encrypted data: ", end="\n")
    print(encrypted_data)

    initial_merkle_tree = generate_merkle_tree(data.encode())
    print("--"*50)
    print("Initial merkle root: ",end="\n")
    print(initial_merkle_tree)
    print("--"*50)

    client_socket.send(encrypted_data)
    print("Sent encrypted data")

    # Receive acknowledgment from the server
    ack = client_socket.recv(1024).decode()
    print("Server acknowledgment: ", ack)

    # Check if the server acknowledged the data
    if ack == "Data Received":
        print("Server received the data successfully.")

        # Request the server to send back the data
        client_socket.send("REQUEST_DATA".encode())
        print("Requested data from server")

        # Receive the data from the server
        received_data = client_socket.recv(8192)
        print("Received data from server. Length: ", len(received_data))

        # Save the received data to a file
        with open('output/ServerToClient.txt', 'wb') as file:
            file.write(received_data)

        print("Saved received data to 'ServerToClient.txt'")

        decrypted_data = simple_encrypt(received_data)

        """# Decrypt the received data
        print("--"*50)
        print("Data sent back by server: ", end="\n")
        print(decrypted_data.decode())
        print("--"*50)"""

        with open('output/DecryptedFile.txt', 'wb') as file:
            file.write(decrypted_data)

        # Generate a new Merkle tree from the decrypted data
        new_merkle_tree = generate_merkle_tree(decrypted_data)
        print("--"*50)
        print("New merkle root: ",end="\n")
        print(new_merkle_tree)
        print("--"*50)

        if new_merkle_tree == initial_merkle_tree:
            print("Merkle tree verification successful :) yayyy ")
            print("File integrity preserved")
        else:
            print("Merkle tree verification failed. :( moreDebugging")
            print("File integrity compromised")

    client_socket.close()

if __name__ == '__main__':
    client_pgm()


Encrypted data: 
b'\x1b\x1d\x19\x0e\x19\x1b\x19\x1b\x1b\x1b\x0e\x19\x1b\x1b\x1d\x19\x19\x0e\x19\x1b\x1d\x1b\x1b\x1b\x1d\x1b\x1d\x19\x0e\x1d\x1b\x19\x1b\x0e\x19\x19\x0e\x1b\x1d\x19\x0e\x19\x0e\x0e\x19\x19\x19\x1d\x1d\x1b\x1b\x1b\x1b\x1b\x19\x0e\x19\x1d\x1b\x1b\x0e\x1d\x0e\x19\x1d\x19\x19\x19\x0e\x1dP\x19\x19\x1d\x0e\x0e\x19\x19\x0e\x1d\x1d\x1d\x1d\x0e\x0e\x1d\x1d\x0e\x1d\x1b\x19\x1b\x1d\x1d\x0e\x19\x0e\x1d\x1d\x0e\x19\x1b\x0e\x19\x1d\x19\x1b\x19\x1d\x1d\x19\x1d\x1d\x19\x1b\x1d\x19\x0e\x19\x19\x0e\x19\x1b\x19\x19\x0e\x1d\x1d\x1b\x0e\x0e\x0e\x1b\x1d\x1b\x1b\x1d\x1b\x1d\x19\x0eP\x1d\x1d\x19\x1d\x0e\x19\x19\x19\x19\x1d\x19\x19\x19\x1d\x19\x19\x19\x1b\x1b\x1d\x19\x19\x0e\x0e\x0e\x1b\x1b\x1b\x19\x0e\x19\x0e\x19\x1d\x0e\x19\x0e\x1d\x19\x19\x1b\x1d\x1b\x1b\x19\x19\x19\x1d\x19\x19\x1b\x1b\x19\x0e\x19\x0e\x19\x19\x1b\x1d\x1d\x1d\x0e\x1b\x19\x1b\x1b\x1b\x1d\x0eP\x1b\x19\x1b\x1d\x19\x1b\x1d\x1d\x1d\x1b\x19\x1d\x19\x1d\x1d\x1d\x0e\x1d\x1d\x1b\x1d\x19\x19\x19\x0e\x0e\x19\x19\x1b\x1b\x1d\x19\x1d\x1d\x